In [0]:
sm_raw_data = spark.table('social_media_log')

In [0]:
ip_like_count_in_one_second = spark.sql('''
SELECT ip, count(*) OVER (
    PARTITION BY ip
    ORDER BY timestamp
    RANGE BETWEEN INTERVAL 1 SECOND PRECEDING AND CURRENT ROW
) as ip_count
FROM social_media_log
WHERE action = "like"
''')

In [0]:
display(ip_like_count_in_one_second)

In [0]:
sus_ips = ip_like_count_in_one_second.select('ip').where(ip_like_count_in_one_second.ip_count > 5).distinct()
sus_ips.write.mode('overwrite').saveAsTable('suspicious_ips')
print(sus_ips.count())

In [0]:
display(sus_ips)

In [0]:
sm_cleaned_data = sm_raw_data.join(
    sus_ips,
    on='IP',
    how='leftanti'
)
sm_cleaned_data.write.mode('overwrite').saveAsTable('social_media_clean')
print(sm_cleaned_data.count())